# Zadanie 15

In [1]:
import numpy as np
from itertools import product
import random
from src.eca import ECA

In [88]:
def insert_spatial_gaps(diagram):
    for row in diagram:
        index = random.randint(0, len(row) - 1)
        row[index] = -1
    return np.array(diagram)


def get_rule_from_diagram(diagram: (list, np.ndarray), r: int):
    rule_dict = {tup: None for tup in list(product([1, 0], repeat=1 + 2 * r))}

    for current_row, next_row in zip(diagram, diagram[1:]):
        extended_row = np.array([*current_row[-r:], *current_row, *current_row[:r]])
        neighborhoods = np.lib.stride_tricks.sliding_window_view(extended_row, window_shape=1 + 2 * r)

        for neighborhood, output in zip(neighborhoods, next_row):
            if -1 not in neighborhood and output != -1:
                rule_dict[tuple(neighborhood)] = output

        if None not in rule_dict.values():
            break

    if None in rule_dict.values():
        return 'Rule not found!'

    rule_number = ''.join(str(rule_dict[tup]) for tup in product([1, 0], repeat=3))
    return int(rule_number, 2)


def ca_identify(observation: np.ndarray):
    neighborhoods = np.roll(observation, -1, axis=1) + observation * 2 + 4 * np.roll(observation, 1, axis=1)
    return neighborhoods


# better
def identify_rule(spacetime_diagram: np.ndarray) -> int:
    counter = np.zeros((9, 3), dtype=int)
    diagram = spacetime_diagram.copy()

    diagram[diagram == -1] = -100
    identified = ca_identify(diagram)[:-1]
    identified[identified < 0] = 8
    diagram = diagram[1:]
    diagram[diagram < 0] = 2

    np.add.at(counter, (identified.flatten(), diagram.flatten()), 1)
    counter = counter[::-1][1:]
    counter = counter[:, :2]

    bits = [np.argmax(sublist) for sublist in counter]

    return int(''.join(str(bit) for bit in bits), 2)

# Identifying Rules From Spacetime Diagram

In [89]:
automaton = ECA.init_random(10)
automaton.evolve(111, 10)
diagram = automaton.history
diagram = insert_spatial_gaps(diagram)

In [90]:
print(f'Reguła: {get_rule_from_diagram(diagram, 1)}')

Reguła: 111


In [91]:
print(f'Reguła: {identify_rule(diagram)}')

Reguła: 111
